In [67]:
import os.path
import math
import re
import tensorflow as tf
import numpy as np
import facenet
from scipy.spatial import distance
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN, MeanShift, estimate_bandwidth, Birch
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import time
%matplotlib inline

In [184]:
def get_model_filenames(model_dir):
    files = os.listdir(model_dir)
    meta_files = [s for s in files if s.endswith('.meta')]
    if len(meta_files)==0:
        raise ValueError('No meta file found in the model directory (%s)' % model_dir)
    elif len(meta_files)>1:
        raise ValueError('There should not be more than one meta file in the model directory (%s)' % model_dir)
    meta_file = meta_files[0]
    meta_files = [s for s in files if '.ckpt' in s]
    max_step = -1
    for f in files:
        step_str = re.match(r'(^model-[\w\- ]+.ckpt-(\d+))', f)
        if step_str is not None and len(step_str.groups())>=2:
            step = int(step_str.groups()[1])
            if step > max_step:
                max_step = step
                ckpt_file = step_str.groups()[0]
    return meta_file, ckpt_file

def load_model(model, sess):
    model_exp = os.path.expanduser(model)
    print('Model directory: %s' % model_exp)
    meta_file, ckpt_file = get_model_filenames(model_exp)

    print('Metagraph file: %s' % meta_file)
    print('Checkpoint file: %s' % ckpt_file)

    saver = tf.train.import_meta_graph(os.path.join(model_exp, meta_file))
    saver.restore(sess, os.path.join(model_exp, ckpt_file))
    
def forward(paths, batch_size):
    nrof_images = len(paths)
    nrof_batches = int(math.ceil(1.0*nrof_images / batch_size))
    emb_array = np.zeros((nrof_images, embedding_size))

    for i in range(nrof_batches):    
        start_index = i*batch_size
        end_index = min((i+1)*batch_size, nrof_images)
        paths_batch = paths[start_index:end_index]
        images = facenet.load_data(paths_batch, False, False, image_size)
        feed_dict = { images_placeholder:images, phase_train_placeholder:False }
        emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)    
        print("batch number:", i)
    print("Complete")
    
    return emb_array

def printDistance(emb_array, label_list, end=10):
    threshold = 1.0
    lng = len(emb_array[:end])
    print('Distance matrix')
    print('    ', end='')
    for i in range(lng):
        print('    %3d     ' % i, end='')
    print('')
    for i in range(lng):
        print('%1d  ' % i, end='')
        for j in range(lng):
            dist = np.linalg.norm(emb_array[i,:] - emb_array[j,:])
            print('  %1.4f' % dist, end='')
            if (label_list[i] == label_list[j]) and (dist <= threshold):
                print("(TP)", end='')
            elif (label_list[i] != label_list[j]) and (dist > threshold):
                print("(TN)", end='')
            elif (label_list[i] == label_list[j]) and (dist > threshold):
                print("\033[93m(FN)\033[0m", end='')
            elif (label_list[i] != label_list[j]) and (dist <= threshold):
                print("\033[93m(FP)\033[0m", end='')
        print('')
        
def printData(labels, paths, number):
    for i in range(number):
        if i > 0 and labels[i] != labels[i-1]:
            print('')
        print('{0:3d} {1:3d}   {2:s}'.format(i, labels[i], paths[i].split("\\")[1]))
        # print('{0:3d}   {1:s}'.format(labels[i], paths[i].split("\\")[1]))
        
def PCAtransform(emb_array, n_components = 10):
    pca = PCA(n_components, whiten=True)
    pca.fit(emb_array)
    return pca.transform(emb_array), pca

def splitTrainTest(data_set, percent_train):
    paths, labels_list = facenet.get_image_paths_and_labels(data_set)
    
    lng = len(paths)
    lng_train = int(lng * percent_train / 100);
    lng_test = lng - lng_train
    
    train_paths = paths[0:lng_train]
    test_paths = paths[lng_train:]
    
    train_labels = labels_list[0:lng_train]
    test_labels = labels_list[lng_train:]
    
    return train_paths, train_labels, test_paths, test_labels

def preprocessing(X, num_components = 50):
    num_features = X.shape[1]
    
    X = X - np.mean(X, axis=0) # centering
    X = X / X.std(axis=0) # standardization

    # PCA
    cov = np.cov(X.T)
    eig_vals, eig_vecs = np.linalg.eig(cov)

    for ev in eig_vecs:
        np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))

    # Make a list of (eigenvalue, eigenvector) tuples
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

    # Sort the (eigenvalue, eigenvector) tuples from high to low
    eig_pairs.sort()
    eig_pairs.reverse()

    pca_matrix = eig_pairs[0][1].reshape(num_features, 1)

    for i in range(1, num_components):
        pca_matrix = np.hstack((pca_matrix,
                                eig_pairs[i][1].reshape(num_features, 1)))

    X = X.dot(pca_matrix)

    # Whitening
    X = X / np.sqrt(eig_vals[:num_components])
    
    return X
    

def trainFisher(emb_array, y):
    emb_del = np.empty_like(emb_array)
    np.copyto(emb_del, emb_array)
    
    emb_array = np.vstack((emb_array, y))
    
    E = np.linalg.inv(np.cov(emb_array.T))
    D = (y - (1 / (len(emb_array) - 1) * np.sum(emb_del)))
    
    w = np.dot(E, D)
    c = np.dot(w / np.linalg.norm(w), y)
    
    return c, w

def trainFisherWithNum(emb_array, sample_number):    
    y = emb_array[sample_number]
    emb_del = np.delete(emb_array, (sample_number), axis=0)
    
    E = np.linalg.inv(np.cov(emb_array.T))
    D = (y - (1 / (len(emb_array) - 1) * np.sum(emb_del)))
    
    w = np.dot(E, D)
    c = np.dot(w / np.linalg.norm(w), y)
    
    return c, w   

def Fisher(w, c, sample):
    return np.dot(w / np.linalg.norm(w), sample) - c

In [3]:
sess = tf.Session()
data_set = facenet.get_dataset("../../datasets/lfw/lfw_mtcnnpy_160/")
train_paths, train_labels, test_paths, test_labels = splitTrainTest(data_set, 100)
load_model("20170511-185253", sess)

images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

image_size = 160
embedding_size = embeddings.get_shape()[1]

Model directory: 20170511-185253
Metagraph file: model-20170511-185253.meta
Checkpoint file: model-20170511-185253.ckpt-80000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from 20170511-185253\model-20170511-185253.ckpt-80000


In [12]:
emb_array = np.load('embeddings.npy')

In [15]:
t = emb_array

# Centering
tc = t - np.mean(t, axis=0)

# Standartization
ts = tc / tc.std(axis=0)

# PCA
X = ts

num_components = 50

cov = np.cov(X.T)
eig_vals, eig_vecs = np.linalg.eig(cov)

for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
print('Everything is ok!')

# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

pca_matrix = eig_pairs[0][1].reshape(128, 1)

for i in range(1, num_components):
    pca_matrix = np.hstack((pca_matrix,
                            eig_pairs[i][1].reshape(128, 1)))

X_pca = X.dot(pca_matrix)

# Whitening
X_whit = X_pca / np.sqrt(eig_vals[:num_components])

Everything ok!


**Fisher's discriminant**

In [425]:
bush_label = 5746
idx, = np.where(train_labels == bush_label)

X_work = np.delete(X_whit, idx, axis=0)
c, w = trainFisher(X_work, X_whit[idx[0]])

lng = len(X_whit)
summ = 0
thr = 0.7
answers = []

for i in range(lng):
    l = Fisher(w, thr, X_whit[i]/np.linalg.norm(X_whit[i]))
    answers.append(l)
    
answers = np.asarray(answers)
true_labels = np.array(train_labels == bush_label, dtype=int)
prediction = np.array(answers > 0, dtype=int)

print("Precision: ", metrics.precision_score(true_labels, prediction))
print("Recall:    ", metrics.recall_score(true_labels, prediction))

Precision:  0.5
Recall:     1.0


**FN and FP**

In [436]:
fn = len(prediction[idx][prediction[idx] != true_labels[idx]])
fnr = fn / len(true_labels[idx])

print("Elements:             ", len(prediction[idx]))
print("False Negatives:      ", fn)
print("False Negatives Rate: ", fnr)
print("")

idx2, = np.where(train_labels != bush_label)
fp = len(prediction[idx2][prediction[idx2] != true_labels[idx2]])
fpr = fp / len(true_labels[idx2])

print("Elements:             ", len(prediction[idx2]))
print("False Positives:      ", fp)
print("False Positives Rate: ", fpr)

Elements:              1
False Negatives:       0
False Negatives Rate:  0.0

Elements:              13232
False Positives:       1
False Positives Rate:  7.557436517533252e-05


**Array of FN and FP**

In [393]:
labels = np.unique(train_labels)

fps = []
fns = []
info = []

lng = len(X_whit)
thr = 0.7

for lb in labels:
    bush_label = lb
    answers = []
    idx, = np.where(train_labels == bush_label)

    X_work = np.delete(X_whit, idx, axis=0)
    c, w = trainFisher(X_work, X_whit[idx[0]])
    
    for i in range(lng):
        l = Fisher(w, thr, X_whit[i]/np.linalg.norm(X_whit[i]))
        answers.append(l)
    
    answers = np.asarray(answers)
    true_labels = np.array(train_labels == bush_label, dtype=int)
    prediction = np.array(answers > 0, dtype=int)
    
    fn = len(prediction[idx][prediction[idx] != true_labels[idx]])
    fnr = fn / len(true_labels[idx])
    
    idx2, = np.where(train_labels != bush_label)
    fp = len(prediction[idx2][prediction[idx2] != true_labels[idx2]])
    fpr = fp / len(true_labels[idx2])
    
    fns.append(fnr)
    fps.append(fpr)
    info.append([len(prediction[idx]), fn, fnr, len(prediction[idx2]), fp, fpr])
    
    print(bush_label, "fnr: ", fnr, "fpr: ", fpr)

0 fnr:  0.0 fpr:  0.0
1 fnr:  0.0 fpr:  0.0
2 fnr:  0.0 fpr:  0.0
3 fnr:  0.0 fpr:  0.0
4 fnr:  0.0 fpr:  0.0
5 fnr:  0.25 fpr:  0.0
6 fnr:  0.0 fpr:  0.0
7 fnr:  0.5 fpr:  0.0
8 fnr:  0.0 fpr:  0.0
9 fnr:  0.0 fpr:  0.0
10 fnr:  0.0 fpr:  0.0
11 fnr:  0.0 fpr:  0.0
12 fnr:  0.0 fpr:  0.0
13 fnr:  0.0 fpr:  0.0
14 fnr:  0.75 fpr:  0.0
15 fnr:  0.0 fpr:  0.0
16 fnr:  0.0 fpr:  0.0
17 fnr:  0.0 fpr:  0.0
18 fnr:  0.0 fpr:  0.0
19 fnr:  0.0 fpr:  0.0
20 fnr:  0.3684210526315789 fpr:  7.567731194187983e-05
21 fnr:  0.0 fpr:  0.0
22 fnr:  0.3333333333333333 fpr:  0.0
23 fnr:  0.0 fpr:  0.0
24 fnr:  0.0 fpr:  0.0
25 fnr:  0.5 fpr:  0.0
26 fnr:  0.6666666666666666 fpr:  0.0
27 fnr:  0.0 fpr:  0.0
28 fnr:  0.0 fpr:  0.0
29 fnr:  0.0 fpr:  0.0
30 fnr:  0.0 fpr:  0.0
31 fnr:  0.0 fpr:  0.0
32 fnr:  0.0 fpr:  0.0
33 fnr:  0.0 fpr:  0.0
34 fnr:  0.0 fpr:  0.0
35 fnr:  0.0 fpr:  0.0
36 fnr:  0.5 fpr:  0.0
37 fnr:  0.5 fpr:  0.0
38 fnr:  0.0 fpr:  0.0
39 fnr:  0.0 fpr:  0.0
40 fnr:  0.0 fpr:  0.0
41

326 fnr:  0.0 fpr:  0.0
327 fnr:  0.6666666666666666 fpr:  0.0
328 fnr:  0.0 fpr:  0.0
329 fnr:  0.5 fpr:  0.0
330 fnr:  0.5 fpr:  0.0
331 fnr:  0.0 fpr:  0.0
332 fnr:  0.0 fpr:  0.0
333 fnr:  0.0 fpr:  0.0
334 fnr:  0.0 fpr:  0.0
335 fnr:  0.0 fpr:  0.0
336 fnr:  0.0 fpr:  0.0
337 fnr:  0.0 fpr:  0.0
338 fnr:  0.0 fpr:  0.0
339 fnr:  0.0 fpr:  0.0
340 fnr:  0.0 fpr:  0.0
341 fnr:  0.0 fpr:  0.0
342 fnr:  0.0 fpr:  0.0
343 fnr:  0.5 fpr:  0.0
344 fnr:  0.0 fpr:  0.0
345 fnr:  0.0 fpr:  0.0
346 fnr:  0.0 fpr:  0.0
347 fnr:  0.0 fpr:  0.0
348 fnr:  0.0 fpr:  0.0
349 fnr:  0.0 fpr:  0.0
350 fnr:  0.0 fpr:  0.0
351 fnr:  0.0 fpr:  0.0
352 fnr:  0.0 fpr:  0.0
353 fnr:  0.0 fpr:  0.0
354 fnr:  0.6 fpr:  0.0
355 fnr:  0.0 fpr:  0.0
356 fnr:  0.0 fpr:  0.0
357 fnr:  0.0 fpr:  0.0
358 fnr:  0.0 fpr:  0.0
359 fnr:  0.25 fpr:  0.0
360 fnr:  0.3333333333333333 fpr:  0.0
361 fnr:  0.0 fpr:  0.0003023660140600197
362 fnr:  0.0 fpr:  0.0
363 fnr:  0.5 fpr:  0.0
364 fnr:  0.0 fpr:  0.0
365 fnr:  0.0 f

646 fnr:  0.0 fpr:  0.0
647 fnr:  0.0 fpr:  0.0
648 fnr:  0.0 fpr:  0.0
649 fnr:  0.0 fpr:  0.0
650 fnr:  0.0 fpr:  0.0
651 fnr:  0.0 fpr:  0.0
652 fnr:  0.0 fpr:  0.0
653 fnr:  0.0 fpr:  0.0
654 fnr:  0.0 fpr:  0.0
655 fnr:  0.0 fpr:  0.0
656 fnr:  0.0 fpr:  0.0
657 fnr:  0.0 fpr:  0.0
658 fnr:  0.0 fpr:  0.0
659 fnr:  0.0 fpr:  0.0
660 fnr:  0.6666666666666666 fpr:  0.0
661 fnr:  0.0 fpr:  0.0
662 fnr:  0.0 fpr:  0.0
663 fnr:  0.0 fpr:  0.0
664 fnr:  0.0 fpr:  0.0
665 fnr:  0.0 fpr:  0.0
666 fnr:  0.0 fpr:  0.0
667 fnr:  0.0 fpr:  0.0
668 fnr:  0.0 fpr:  0.0
669 fnr:  0.0 fpr:  0.0
670 fnr:  0.0 fpr:  0.0
671 fnr:  0.0 fpr:  0.0
672 fnr:  0.0 fpr:  0.0
673 fnr:  0.0 fpr:  0.0
674 fnr:  0.0 fpr:  0.0
675 fnr:  0.0 fpr:  0.0
676 fnr:  0.0 fpr:  0.0
677 fnr:  0.0 fpr:  0.0
678 fnr:  0.0 fpr:  0.0
679 fnr:  0.0 fpr:  0.0
680 fnr:  0.0 fpr:  0.0
681 fnr:  0.0 fpr:  0.0
682 fnr:  0.0 fpr:  0.0
683 fnr:  0.0 fpr:  0.0
684 fnr:  0.0 fpr:  0.0
685 fnr:  0.0 fpr:  0.0
686 fnr:  0.0 fpr:  0.0
6

970 fnr:  0.0 fpr:  0.0
971 fnr:  0.0 fpr:  0.0
972 fnr:  0.6 fpr:  0.0
973 fnr:  0.0 fpr:  0.0
974 fnr:  0.0 fpr:  0.0
975 fnr:  0.0 fpr:  0.0
976 fnr:  0.3333333333333333 fpr:  0.0
977 fnr:  0.0 fpr:  0.0
978 fnr:  0.0 fpr:  0.0
979 fnr:  0.0 fpr:  0.0
980 fnr:  0.0 fpr:  0.0
981 fnr:  0.0 fpr:  0.0
982 fnr:  0.75 fpr:  0.0
983 fnr:  0.0 fpr:  0.0
984 fnr:  0.0 fpr:  0.0
985 fnr:  0.25 fpr:  0.0
986 fnr:  0.0 fpr:  0.0
987 fnr:  0.0 fpr:  0.0
988 fnr:  0.0 fpr:  0.0
989 fnr:  0.0 fpr:  0.0
990 fnr:  0.0 fpr:  0.0
991 fnr:  0.0 fpr:  0.0
992 fnr:  0.0 fpr:  0.0
993 fnr:  0.0 fpr:  0.0
994 fnr:  0.0 fpr:  0.0
995 fnr:  0.0 fpr:  0.0
996 fnr:  0.0 fpr:  0.0
997 fnr:  0.5 fpr:  0.0
998 fnr:  0.0 fpr:  0.0
999 fnr:  0.0 fpr:  7.557436517533252e-05
1000 fnr:  0.0 fpr:  0.0
1001 fnr:  0.6666666666666666 fpr:  0.0
1002 fnr:  0.0 fpr:  0.0
1003 fnr:  0.5 fpr:  0.0
1004 fnr:  0.0 fpr:  0.0
1005 fnr:  0.0 fpr:  0.0
1006 fnr:  0.0 fpr:  0.0
1007 fnr:  0.2 fpr:  0.0
1008 fnr:  0.6666666666666666 

1286 fnr:  0.0 fpr:  0.0
1287 fnr:  0.0 fpr:  0.0
1288 fnr:  0.0 fpr:  0.0
1289 fnr:  0.0 fpr:  0.0
1290 fnr:  0.0 fpr:  0.0
1291 fnr:  0.0 fpr:  0.0
1292 fnr:  0.0 fpr:  0.0
1293 fnr:  0.0 fpr:  0.0
1294 fnr:  0.0 fpr:  0.0
1295 fnr:  0.5 fpr:  0.0
1296 fnr:  0.25 fpr:  0.0
1297 fnr:  0.0 fpr:  0.0
1298 fnr:  0.0 fpr:  0.0
1299 fnr:  0.5 fpr:  0.0
1300 fnr:  0.0 fpr:  0.0
1301 fnr:  0.0 fpr:  0.0
1302 fnr:  0.0 fpr:  0.0
1303 fnr:  0.0 fpr:  0.0
1304 fnr:  0.0 fpr:  0.0
1305 fnr:  0.0 fpr:  0.0
1306 fnr:  0.3333333333333333 fpr:  0.0
1307 fnr:  0.0 fpr:  0.0
1308 fnr:  0.5 fpr:  0.0
1309 fnr:  0.42857142857142855 fpr:  0.0
1310 fnr:  0.0 fpr:  0.0
1311 fnr:  0.0 fpr:  0.0
1312 fnr:  0.5 fpr:  0.0
1313 fnr:  0.0 fpr:  0.0
1314 fnr:  0.4 fpr:  0.0
1315 fnr:  0.0 fpr:  0.0
1316 fnr:  0.0 fpr:  0.0
1317 fnr:  0.0 fpr:  0.0
1318 fnr:  0.75 fpr:  0.0
1319 fnr:  0.0 fpr:  0.0
1320 fnr:  0.5 fpr:  0.0
1321 fnr:  0.0 fpr:  0.0
1322 fnr:  0.0 fpr:  0.0
1323 fnr:  0.0 fpr:  0.0
1324 fnr:  0.0 fp

...

4385 fnr:  0.0 fpr:  0.0
4386 fnr:  0.0 fpr:  0.0
4387 fnr:  0.0 fpr:  0.0
4388 fnr:  0.5 fpr:  0.0
4389 fnr:  0.0 fpr:  0.0
4390 fnr:  0.5 fpr:  0.0
4391 fnr:  0.0 fpr:  0.0
4392 fnr:  0.0 fpr:  0.0
4393 fnr:  0.0 fpr:  0.0
4394 fnr:  0.0 fpr:  0.0
4395 fnr:  0.5 fpr:  0.0
4396 fnr:  0.0 fpr:  0.0
4397 fnr:  0.0 fpr:  0.0
4398 fnr:  0.0 fpr:  0.0
4399 fnr:  0.0 fpr:  7.557436517533252e-05
4400 fnr:  0.0 fpr:  0.0
4401 fnr:  0.0 fpr:  0.0
4402 fnr:  0.0 fpr:  0.0
4403 fnr:  0.7333333333333333 fpr:  0.0
4404 fnr:  0.0 fpr:  0.0
4405 fnr:  0.0 fpr:  0.0
4406 fnr:  0.0 fpr:  0.0
4407 fnr:  0.0 fpr:  0.0
4408 fnr:  0.0 fpr:  0.0
4409 fnr:  0.0 fpr:  0.0
4410 fnr:  0.0 fpr:  0.0
4411 fnr:  0.0 fpr:  0.0
4412 fnr:  0.0 fpr:  0.0
4413 fnr:  0.0 fpr:  0.0
4414 fnr:  0.0 fpr:  0.0
4415 fnr:  0.0 fpr:  0.0
4416 fnr:  0.6666666666666666 fpr:  0.0
4417 fnr:  0.0 fpr:  0.0
4418 fnr:  0.0 fpr:  0.0
4419 fnr:  0.0 fpr:  0.0
4420 fnr:  0.0 fpr:  0.0
4421 fnr:  0.0 fpr:  0.0
4422 fnr:  0.0 fpr:  0.0
44

4691 fnr:  0.0 fpr:  0.0
4692 fnr:  0.0 fpr:  0.0
4693 fnr:  0.0 fpr:  0.0
4694 fnr:  0.0 fpr:  0.0
4695 fnr:  0.0 fpr:  0.0
4696 fnr:  0.0 fpr:  0.0
4697 fnr:  0.0 fpr:  0.0
4698 fnr:  0.0 fpr:  0.0
4699 fnr:  0.0 fpr:  0.0
4700 fnr:  0.0 fpr:  0.0
4701 fnr:  0.0 fpr:  0.0
4702 fnr:  0.0 fpr:  0.0
4703 fnr:  0.0 fpr:  0.0
4704 fnr:  0.375 fpr:  0.0
4705 fnr:  0.0 fpr:  0.0
4706 fnr:  0.0 fpr:  0.0
4707 fnr:  0.6666666666666666 fpr:  0.0
4708 fnr:  0.0 fpr:  0.0
4709 fnr:  0.0 fpr:  0.0
4710 fnr:  0.0 fpr:  0.0
4711 fnr:  0.0 fpr:  0.0
4712 fnr:  0.0 fpr:  0.0
4713 fnr:  0.0 fpr:  0.0
4714 fnr:  0.0 fpr:  0.0
4715 fnr:  0.0 fpr:  0.0
4716 fnr:  0.8571428571428571 fpr:  0.0
4717 fnr:  0.0 fpr:  0.0
4718 fnr:  0.0 fpr:  0.0
4719 fnr:  0.0 fpr:  0.0
4720 fnr:  0.0 fpr:  0.0
4721 fnr:  0.5 fpr:  0.0
4722 fnr:  0.0 fpr:  0.0
4723 fnr:  0.0 fpr:  0.0
4724 fnr:  0.0 fpr:  0.0
4725 fnr:  0.0 fpr:  0.0
4726 fnr:  0.0 fpr:  0.0
4727 fnr:  0.3333333333333333 fpr:  0.0
4728 fnr:  0.0 fpr:  0.0
472

5001 fnr:  0.0 fpr:  0.0
5002 fnr:  0.0 fpr:  0.0
5003 fnr:  0.0 fpr:  7.557436517533252e-05
5004 fnr:  0.0 fpr:  0.0
5005 fnr:  0.0 fpr:  0.0
5006 fnr:  0.0 fpr:  0.0
5007 fnr:  0.0 fpr:  0.0
5008 fnr:  0.0 fpr:  0.0
5009 fnr:  0.5 fpr:  0.0
5010 fnr:  0.0 fpr:  0.0
5011 fnr:  0.0 fpr:  0.0
5012 fnr:  0.0 fpr:  0.0
5013 fnr:  0.0 fpr:  0.0
5014 fnr:  0.0 fpr:  0.0
5015 fnr:  0.0 fpr:  0.0
5016 fnr:  0.875 fpr:  0.0
5017 fnr:  0.0 fpr:  0.0
5018 fnr:  0.0 fpr:  0.0
5019 fnr:  0.0 fpr:  0.0
5020 fnr:  0.0 fpr:  0.0
5021 fnr:  0.875 fpr:  0.0
5022 fnr:  0.0 fpr:  0.0
5023 fnr:  0.0 fpr:  7.557436517533252e-05
5024 fnr:  0.0 fpr:  7.557436517533252e-05
5025 fnr:  0.0 fpr:  0.0
5026 fnr:  0.0 fpr:  0.0
5027 fnr:  0.0 fpr:  0.0
5028 fnr:  0.0 fpr:  0.0
5029 fnr:  0.0 fpr:  0.0
5030 fnr:  0.0 fpr:  0.0
5031 fnr:  0.0 fpr:  7.557436517533252e-05
5032 fnr:  0.0 fpr:  0.0
5033 fnr:  0.0 fpr:  0.0
5034 fnr:  0.0 fpr:  0.0
5035 fnr:  0.75 fpr:  0.0
5036 fnr:  0.5 fpr:  0.0
5037 fnr:  0.6666666666

5309 fnr:  0.0 fpr:  0.0
5310 fnr:  0.8 fpr:  0.0
5311 fnr:  0.0 fpr:  0.0
5312 fnr:  0.0 fpr:  0.0
5313 fnr:  0.0 fpr:  0.0
5314 fnr:  0.0 fpr:  0.0
5315 fnr:  0.0 fpr:  0.0
5316 fnr:  0.5 fpr:  0.0
5317 fnr:  0.3333333333333333 fpr:  0.0
5318 fnr:  0.0 fpr:  0.0
5319 fnr:  0.0 fpr:  0.0
5320 fnr:  0.6666666666666666 fpr:  0.0
5321 fnr:  0.0 fpr:  0.0
5322 fnr:  0.5 fpr:  0.0
5323 fnr:  0.5 fpr:  0.0
5324 fnr:  0.0 fpr:  0.0
5325 fnr:  0.0 fpr:  0.0
5326 fnr:  0.0 fpr:  0.0
5327 fnr:  0.0 fpr:  0.0
5328 fnr:  0.5 fpr:  0.0
5329 fnr:  0.0 fpr:  0.0
5330 fnr:  0.0 fpr:  0.0
5331 fnr:  0.0 fpr:  0.0
5332 fnr:  0.0 fpr:  0.0
5333 fnr:  0.0 fpr:  0.0
5334 fnr:  0.0 fpr:  0.0
5335 fnr:  0.5 fpr:  0.0
5336 fnr:  0.0 fpr:  0.0
5337 fnr:  0.0 fpr:  0.0
5338 fnr:  0.4444444444444444 fpr:  0.0
5339 fnr:  0.6666666666666666 fpr:  0.0
5340 fnr:  0.0 fpr:  0.0
5341 fnr:  0.0 fpr:  0.0
5342 fnr:  0.0 fpr:  0.0
5343 fnr:  0.0 fpr:  0.0
5344 fnr:  0.0 fpr:  0.0
5345 fnr:  0.0 fpr:  0.0
5346 fnr:  0.0 

5615 fnr:  0.0 fpr:  0.0
5616 fnr:  0.0 fpr:  0.0
5617 fnr:  0.0 fpr:  0.0
5618 fnr:  0.0 fpr:  0.0
5619 fnr:  0.5 fpr:  0.0
5620 fnr:  0.0 fpr:  0.0
5621 fnr:  0.0 fpr:  0.0
5622 fnr:  0.75 fpr:  0.0
5623 fnr:  0.5 fpr:  0.0
5624 fnr:  0.0 fpr:  0.0
5625 fnr:  0.0 fpr:  0.0
5626 fnr:  0.7142857142857143 fpr:  0.0
5627 fnr:  0.42857142857142855 fpr:  0.0
5628 fnr:  0.0 fpr:  0.0
5629 fnr:  0.0 fpr:  0.0
5630 fnr:  0.5 fpr:  0.0
5631 fnr:  0.0 fpr:  0.0
5632 fnr:  0.0 fpr:  0.0
5633 fnr:  0.0 fpr:  0.0
5634 fnr:  0.0 fpr:  0.0
5635 fnr:  0.2 fpr:  0.0
5636 fnr:  0.5 fpr:  0.0
5637 fnr:  0.0 fpr:  0.0
5638 fnr:  0.0 fpr:  0.0
5639 fnr:  0.0 fpr:  0.0
5640 fnr:  0.0 fpr:  0.0
5641 fnr:  0.0 fpr:  0.0
5642 fnr:  0.0 fpr:  0.0
5643 fnr:  0.0 fpr:  0.0
5644 fnr:  0.0 fpr:  0.0
5645 fnr:  0.0 fpr:  0.0
5646 fnr:  0.0 fpr:  0.0
5647 fnr:  0.0 fpr:  0.0
5648 fnr:  0.0 fpr:  0.0
5649 fnr:  0.0 fpr:  0.0
5650 fnr:  0.0 fpr:  0.0
5651 fnr:  0.0 fpr:  0.0
5652 fnr:  0.0 fpr:  0.0
5653 fnr:  0.0 fpr

In [452]:
np.savetxt("info.csv", np.asarray(info), fmt='%.6f', delimiter=",")

In [459]:
np.asarray(fps).mean()

1.6170258553593387e-06